# Simple Machine Learning with TensorFlow

Regression Example with Linear Regression to Predict Blood Fat Content

In [1]:
import tensorflow as tf

In [2]:
W = tf.Variable(tf.zeros([2,1]), name="weights")
b = tf.Variable(0., name="bias")

def inference(X):
    return tf.matmul(X,W) + b

In [3]:
def loss(X,Y):
    Y_predicted = inference(X)
    return tf.reduce_sum(tf.squared_difference(Y,Y_predicted))

In [4]:
def inputs():
    weights_age = [[84,46],[73,20],[65,52],[70,30],[76,57],[69,25],[63,28],[72,36],[79,57],[75,44],[27,24],[89,31],[89,31],[65,52],[57,23],[59,60],[69,48],[60,34],[79,51],[75,50],[83,34],[59,46],[67,23],[85,37],[85,37],[55,40],[63,30]]
    blood_fat_content = [354,190,405,263,451,302,288,385,402,365,209,290,346,254,395,434,220,374,308,220,311,181,274,303,244]
    return tf.to_float(weights_age),tf.to_float(blood_fat_content)
    #return X,Y

In [5]:
def train(total_loss):
    learning_rate = 0.000000001
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [6]:
def evaluate(sess, X,Y):
    print sess.run(inference([[80.,25.]]))
    print sess.run(inference([[65.,25.]]))

In [7]:
with tf.Session() as sess:
    
    tf.global_variables_initializer().run()
    X,Y = inputs()
    
    total_loss = loss(X,Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        if step % 10 == 0:
            print "loss: ", sess.run([total_loss])
    evaluate(sess, X,Y)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

loss:  [67971088.0]
loss:  [57777528.0]
loss:  [49253456.0]
loss:  [42125456.0]
loss:  [36164864.0]
loss:  [31180514.0]
loss:  [27012482.0]
loss:  [23527100.0]
loss:  [20612542.0]
loss:  [18175336.0]
loss:  [16137287.0]
loss:  [14433036.0]
loss:  [13007900.0]
loss:  [11816177.0]
loss:  [10819627.0]
loss:  [9986295.0]
loss:  [9289443.0]
loss:  [8706721.0]
loss:  [8219435.5]
loss:  [7811960.0]
loss:  [7471219.0]
loss:  [7186288.0]
loss:  [6948020.0]
loss:  [6748774.5]
loss:  [6582160.0]
loss:  [6442833.5]
loss:  [6326327.0]
loss:  [6228902.0]
loss:  [6147432.0]
loss:  [6079306.0]
loss:  [6022336.5]
loss:  [5974695.5]
loss:  [5934859.0]
loss:  [5901546.0]
loss:  [5873688.0]
loss:  [5850395.0]
loss:  [5830914.5]
loss:  [5814625.0]
loss:  [5801003.5]
loss:  [5789612.0]
loss:  [5780084.5]
loss:  [5772120.0]
loss:  [5765456.5]
loss:  [5759888.0]
loss:  [5755230.0]
loss:  [5751332.5]
loss:  [5748075.5]
loss:  [5745350.5]
loss:  [5743072.5]
loss:  [5741168.0]
loss:  [5739572.5]
loss:  [5738239.

Logistic Regression

In [8]:
W_l = tf.Variable(tf.zeros([5,1], name="weights"))
b_l = tf.Variable(0., name="bias")

def combine_inputs(X):
    return tf.matmul(X,W_l) + b

def inference_l(X):
    return tf.sigmoid(combine_inputs(X))

In [53]:
def loss_l(X,Y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=combine_inputs(X),labels=Y))

In [31]:
def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.dirname('__file__') + "/" + file_name])
    
    #os.path.dirname(filename)
    
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    
    decoded = tf.decode_csv(value, record_defaults=record_defaults)
    
    return tf.train.shuffle_batch(decoded, 
                                  batch_size=batch_size,
                                  capacity=batch_size*50,
                                  min_after_dequeue=batch_size
                                 )

In [48]:
def inputs_l():
    passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
    read_csv(100, "train.csv", [[0.0], [0.0], [0], [" "], [" "], [0.0], [0.0], [0.0], [" "], [0.0], [" "], [" "]])
    
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_second_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))
    
    gender= tf.to_float(tf.equal(sex, ["female"]))
    
    features = tf.transpose(tf.stack([is_first_class, is_second_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [100,1])
    
    return features, survived

In [49]:
def train_l(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [50]:
def evaluate(sess, X,Y):
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, y), tf.float32)))

In [51]:
import os

In [54]:
with tf.Session() as sess:
    
    tf.global_variables_initializer().run()
    X,Y = inputs_l()
    
    total_loss = loss_l(X,Y)
    train_op = train_l(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        if step % 10 == 0:
            print "loss: ", sess.run([total_loss])
    evaluate(sess, X,Y)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, /train.csv
	 [[Node: ReaderReadV2 = ReaderReadV2[_device="/job:localhost/replica:0/task:0/cpu:0"](TextLineReaderV2, input_producer)]]

Caused by op u'ReaderReadV2', defined at:
  File "/Users/pauldefusco/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/pauldefusco/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/pauldefusco/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/pauldefusco/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/pauldefusco/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/pauldefusco/anacon

OutOfRangeError: RandomShuffleQueue '_14_shuffle_batch_4/random_shuffle_queue' is closed and has insufficient elements (requested 100, current size 0)
	 [[Node: shuffle_batch_4 = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_FLOAT, DT_INT32, DT_STRING, DT_STRING, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_STRING, DT_FLOAT, DT_STRING, DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch_4/random_shuffle_queue, shuffle_batch_4/n)]]

Caused by op u'shuffle_batch_4', defined at:
  File "/Users/pauldefusco/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/pauldefusco/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/pauldefusco/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/pauldefusco/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", lin